In [80]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelBinarizer, KBinsDiscretizer, OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer, MinMaxScaler, FunctionTransformer, StandardScaler, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
from sklearn import __version__
__version__

'1.5.1'

# EDA

In [2]:
# prompt: code to upload file in pandas

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Assuming the uploaded file is a CSV, you can load it into a pandas DataFrame
import io
df = pd.read_csv(io.BytesIO(uploaded[fn]))
# Replace 'fn' with the actual file name if you have only one file uploaded

# Now you can work with the DataFrame 'df'
df.head()

ModuleNotFoundError: No module named 'google'

In [ ]:
# plt.hist(df.Reporting_District_no, bins=100)
# df.info()

In [ ]:
# df.describe()

### for local runtime

In [3]:
!ls

crime-cast-forecasting-crime-categories.zip  data  main.ipynb


In [4]:
df = pd.read_csv('data/train.csv')

In [5]:
len(df.columns)

22

In [48]:
df.Crime_Category.unique()

array(['Property Crimes', 'Violent Crimes', 'Other Crimes',
       'Crimes against Public Order', 'Fraud and White-Collar Crimes',
       'Crimes against Persons'], dtype=object)

# Preprocessing

#### Spliting train and test sets

In [6]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=1)

In [8]:
X_train.shape, X_test.shape

((16000, 21), (4000, 21))

In [9]:
class CustomModification(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.mlb = MultiLabelBinarizer()
        modus_operandi = X['Modus_Operandi'].map(lambda x: () if type(x) == float else tuple(map(int,x.split(' '))))
        self.mlb.fit(modus_operandi)
        return self

    def transform(self, X, y=None):
        X = X.copy()
        self.clean(X)

        X['reported_year'] = X['Date_Reported'].dt.year
        X['reported_month'] = X['Date_Reported'].dt.month
        X['reported_day'] = X['Date_Reported'].dt.day

        X['occurred_year'] = X['Date_Occurred'].dt.year
        X['occurred_month'] = X['Date_Occurred'].dt.month
        X['occurred_day'] = X['Date_Occurred'].dt.day

        MO = self.mlb.transform(X['Modus_Operandi'])
        X2 = pd.DataFrame(MO, index=X.index)
        X2.columns = X2.columns.map(lambda x: 'MO_' + str(x))
        X = pd.concat([X, X2], axis=1)

        return X

    def clean(self, X):
      X["Latitude"] = X['Latitude'].map(lambda x: np.nan if x == 0 else x)
      X["Longitude"] = X['Longitude'].map(lambda x: np.nan if x == 0 else x)
      X['Date_Reported'] = pd.to_datetime(X['Date_Reported'], format='%m/%d/%Y %H:%M:%S %p')
      X['Date_Occurred'] = pd.to_datetime(X['Date_Occurred'], format='%m/%d/%Y %H:%M:%S %p')
      X.loc[X['Victim_Age'] < 0, 'Victim_Age'] = 0
      X["Modus_Operandi"] = X['Modus_Operandi'].map(lambda x: () if type(x) == float else tuple(map(int,x.split(' '))))



In [10]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist')),
])
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('encoder', KBinsDiscretizer(strategy='kmeans', encode='onehot-dense')),
])

In [11]:
pre_proc = ColumnTransformer([
  ('dropper', 'drop', [0, 1, 4, 5, 8, 10, 11, 16, 18, 20]),
  ('ofe', OrdinalEncoder(), [10]),
  ('ss', StandardScaler(with_mean=True, with_std=True), [6, 12]),
  ('cp1', cat_pipe, [13, 14]),
  ('mms', MinMaxScaler(), list(range(21, 27))),
  ('loc', num_pipe, [2, 3]),
  ('uofe', OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist', min_frequency=0.05), [7,9,15,17,19]),
], remainder='passthrough').set_output(transform='pandas')

In [69]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [12]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.fit_transform(y_test)

In [13]:
lb = LabelBinarizer()
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.fit_transform(y_test)

In [14]:
pipe1 = make_pipeline(CustomModification(), pre_proc, GaussianNB())

In [15]:
pipe2 = make_pipeline(CustomModification(), pre_proc, DecisionTreeClassifier())

In [16]:
pipe3 = make_pipeline(CustomModification(), pre_proc, RandomForestClassifier(random_state=0))

In [17]:
pipe4 = make_pipeline(CustomModification(), pre_proc, LogisticRegression(solver='liblinear',penalty='l1'))
# pipe4 = make_pipeline(CustomModification(), pre_proc, LogisticRegression())

In [76]:
pipe5 = make_pipeline(CustomModification(), pre_proc, SGDClassifier())

In [77]:
print('GaussianNB')
pipe1.fit(X_train, y_train_enc)
trs_gnb = pipe1.score(X_train, y_train_enc)
tes_gnb = pipe1.score(X_test, y_test_enc)
print('Training Score: ', trs_gnb)
print('Testing Score: ', tes_gnb)

GaussianNB
Training Score:  0.3340625
Testing Score:  0.312


In [79]:
print('Decision Tree Classifier')
print('Label Encode Ordinaly')
pipe2.fit(X_train, y_train_enc)
print('Training Score: ', pipe2.score(X_train, y_train_enc))
print('Testing Score: ', pipe2.score(X_test, y_test_enc))
print()
print('Label Binarized')
pipe2.fit(X_train, y_train_bin)
print('Training Score: ', pipe2.score(X_train, y_train_bin))
print('Testing Score: ', pipe2.score(X_test, y_test_bin))

Decision Tree Classifier
Label Encode Ordinaly
Training Score:  1.0
Testing Score:  0.92575

Label Binarized
Training Score:  1.0
Testing Score:  0.9255


In [74]:
print('Random Forest Classifier')

print('Label Encode Ordinaly')
pipe3.fit(X_train, y_train_enc)
print('Training Score: ', pipe3.score(X_train, y_train_enc))
print('Testing Score: ', pipe3.score(X_test, y_test_enc))
print()

print('Label Binarized')
pipe3.fit(X_train, y_train_bin)
print('Training Score: ', pipe3.score(X_train, y_train_bin))
print('Testing Score: ', pipe3.score(X_test, y_test_bin))
print()

print('Cross-validation')
cvl = cross_val_score(pipe3, X, y, cv=skf, scoring='accuracy')
print('CV_score, std: ', cvl.mean(), cvl.std())

Random Forest Classifier
Label Encode Ordinaly
Training Score:  1.0
Testing Score:  0.94625

Label Binarized
Training Score:  1.0
Testing Score:  0.9205

Cross-validation
CV_score, std:  0.94575 0.0028328430948430863


In [68]:
print('Logistic Regression')
pipe4.fit(X_train, y_train_enc)
print('Training Score: ', pipe4.score(X_train, y_train_enc))
print('Testing Score: ', pipe4.score(X_test, y_test_enc))

cvl = cross_val_score(pipe4, X, y, cv=skf, scoring='accuracy')
print('CV_score: ', cvl.mean())

Logistic Regression
Training Score:  0.9555
Testing Score:  0.951
CV_score:  0.9497


In [86]:
print('SGDClassifier')
pipe5.fit(X_train, y_train_enc)
print('Training Score: ', pipe5.score(X_train, y_train_enc))
print('Testing Score: ', pipe5.score(X_test, y_test_enc))

cvl = cross_val_score(pipe4, X, y, cv=skf, scoring='accuracy')
print('CV_score, std: ', np.round(cvl.mean(), 4), np.round(cvl.std(), 4))

SGDClassifier
Training Score:  0.9543125
Testing Score:  0.946
CV_score, std:  0.9496 0.0025


In [23]:
pipe = Pipeline([
    ('custom',  CustomModification()),
    ('pre_process', pre_proc),
    ('classifier', BaggingClassifier(estimator=DecisionTreeClassifier()))
])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

param_grid = {
    'classifier__max_samples': [0.2, 0.6, 1.0],
    'classifier__max_features': [2, 5, 0.5, 1.0],
    'classifier__bootstrap': [True, False],
    'classifier__bootstrap_features': [True, False],
    'classifier__n_estimators': [10, 20, 50],
}

grid_search = GridSearchCV(pipe, param_grid, cv=skf, scoring='accuracy')
grid_search.fit(X, y)

print("Best Parameters: ", grid_search.best_params_)
print("Best Parameters: ", grid_search.best_score_)

Best Parameters:  {'classifier__bootstrap': False, 'classifier__bootstrap_features': True, 'classifier__max_features': 1.0, 'classifier__max_samples': 1.0, 'classifier__n_estimators': 50}


In [51]:
bgcv = make_pipeline(CustomModification(), pre_proc, BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=80,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=False,
    bootstrap_features=True,
    random_state=0
))
cvl = cross_val_score(bgcv, X, y, cv=skf, scoring='accuracy')
m, s = cvl.mean(), cvl.std()
model_report = pd.DataFrame(dict(BaggingClassifier_best=[m, s]), index=['Mean', 'Std'])

In [63]:
model_report.transpose().head(1)

,Mean,Std
Bagging_Classifier_Best,0.9525,0.002302


In [ ]:
pipe = Pipeline([
    ('custom',  CustomModification()),
    ('pre_process', pre_proc),
    ('classifier', SGDClassifier(early_stopping=False, validation_fraction=0.1))
])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

param_grid = {
    'classifier__max_iter': [100, 500],
    'classifier__penalty': ['l2', 'l1', 'elasticnet'],
    'classifier__loss': ['hinge', 'squared_hinge', 'huber', 'modified_huber'],
    'classifier__learning_rate': ['optimal', 'invscaling'],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__l1_ratio': [0.1, 0.15, 0.2],
    'classifier__average': [True, False],
    'classifier__epsilon': [0.05, 0.1, 0.15, 0.2, 0.5],
}

grid_search = GridSearchCV(pipe, param_grid, cv=skf, scoring='accuracy')
grid_search.fit(X, y)

print("Best Parameters: ", grid_search.best_params_)
print("Best Parameters: ", grid_search.best_score_)

In [95]:
grid_search.best_score_, grid_search.best_params_

(0.9430999999999999,
 {'classifier__class_weight': None,
  'classifier__learning_rate': 'optimal',
  'classifier__loss': 'hinge',
  'classifier__max_iter': 100,
  'classifier__penalty': 'elasticnet'})

# Final

In [34]:
model = grid_search.best_estimator_
model.fit(X, y)

Pipeline(steps=[('custom', CustomModification()),
                ('pre_process',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dropper', 'drop',
                                                  [0, 1, 4, 5, 8, 10, 11, 16,
                                                   18, 20]),
                                                 ('ofe', OrdinalEncoder(),
                                                  [10]),
                                                 ('ss', StandardScaler(),
                                                  [6, 12]),
                                                 ('cp1',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='infreque...
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('encoder',
                                                                   KBinsDiscretizer(encode='onehot-dense',
                                                                                    strategy='kmeans'))]),
                                                  [2, 3]),
                                                 ('uofe',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=0.05,
                                                                sparse_output=False),
                                                  [7, 9, 15, 17, 19])])),
                ('classifier',
                 BaggingClassifier(bootstrap=False, bootstrap_features=True,
                                   estimator=DecisionTreeClassifier(),
                                   n_estimators=50))])

In [35]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
import io
Xt = pd.read_csv(io.BytesIO(uploaded[fn]))

ModuleNotFoundError: No module named 'google'

In [36]:
# for local runtime
Xt = pd.read_csv('data/test.csv')

In [37]:
prediction = model.predict(Xt)

In [38]:
# sub = pd.DataFrame(prediction, index=Xt.index, columns=['Crime_Category'])
sub = pd.DataFrame()
sub['Crime_Category'] = prediction
sub.index += 1
sub.index.name = 'ID'
# sub['Crime_Category'] = prediction

In [42]:
sub.to_csv('submission.csv')
files.download('submission.csv')

NameError: name 'files' is not defined

In [43]:
# for local runtime
sub.to_csv('./submission.csv')